## CS310 Natural Language Processing
## Assignment 2. Word2vec Implementation 

**Total points**: 50

Train a word2vec model using the **skip-gram** architecture and **negative sampling**.

You should roughtly follow the structure of the notebook. Add additional cells if you feel needed. 

You can (and you should) re-use the code from *Lab 4 (part 2): Data preparation for implementing word2vec*. 

Make sure your code is readable and well-structured.

### 0. Import Necessary Libraries

In [ ]:
from typing import List
from utils import CorpusReader
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD

### 1. Data Processing

The corpus data is in `lunyu_20chapters.txt`. Use the `CorpusReader` class in `utils.py` to help you.

In [ ]:
# Read raw data in lunyu_20chapters.txt

### YOUR CODE HERE ###
corpus = CorpusReader(inputFileName="lunyu_20chapters.txt", min_count=1)

print(corpus.word2id["子"])
print(corpus.id2word[1])
print(len(corpus.id2word))
### END YOUR CODE ###

In [ ]:
# Re-use the code from lab with necessary modifications

def generate_data(words: List[str], window_size: int, k: int, corpus: CorpusReader):
    """ Generate the training data for word2vec skip-gram model
    Args:
        text: the input text
        window_size: the size of the context window
        k: the number of negative samples
        corpus: the corpus object, providing utilities such as word2id, getNegatives, etc.
    """
    ### YOUR CODE HERE ###
    word_ids = [corpus.word2id[word] for word in words]  # Convert the list of words to a list of word ids

    for center_index, center_id in enumerate(word_ids):
        context_indices = None
        # Iterate over the left context words
        for i in range(max(0, center_index - window_size), center_index):
            context_indices=word_ids[i]
            negative_samples = corpus.getNegatives(center_id, k)
            yield center_id, context_indices, negative_samples

        # Iterate over the right context words
        for i in range(center_index + 1, min(center_index + window_size + 1, len(word_ids))):
            context_indices=word_ids[i]
            negative_samples = corpus.getNegatives(center_id, k)
            yield center_id, context_indices, negative_samples

    ### END YOUR CODE ###

def batchify(data: List, batch_size: int):
    """ Group a stream into batches and yield them as torch tensors.
    Args:
        data: a list of tuples
        batch_size: the batch size 
    Yields:
        a tuple of three torch tensors: center, outside, negative
    """
    assert batch_size < len(data) # data should be long enough
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        if i > len(data) - batch_size: # if the last batch is smaller than batch_size, pad it with the first few data
            batch = batch + data[:i + batch_size - len(data)]
        
        ### YOUR CODE HERE ###
        center_batch = []
        outside_batch = []
        negative_batch = []

        for center, outside, negative in batch:
            center_batch.append(center)
            outside_batch.append(outside)
            negative_batch.append(negative)

        center_tensor = torch.tensor(center_batch, dtype=torch.long)
        outside_tensor = torch.tensor(outside_batch, dtype=torch.long)
        negative_tensor = torch.tensor(negative_batch, dtype=torch.long)

        yield center_tensor, outside_tensor, negative_tensor

        ### END YOUR CODE ###

### 2. Build the Model

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.emb_v = nn.Embedding(vocab_size, emb_size, sparse=True)
        self.emb_u = nn.Embedding(vocab_size, emb_size, sparse=True)

        initrange = 1.0 / self.emb_size # some experience passed down from generation to generation
        nn.init.uniform_(self.emb_v.weight.data, -initrange, initrange) # same outcome as self.emb_v.weight.data.uniform_(-initrange, initrange)
        nn.init.constant_(self.emb_u.weight.data, 0) # same outcome as self.emb_u.weight.data.zero_()

    def forward(self, center, outside, negative):
        """
        Args:
            center: the center word indices (B, )
            outside: the outside word indices (B, )
            negative: the negative word indices (B, k)
        """
        v_c = self.emb_v(center)
        u_o = self.emb_u(outside)
        u_n = self.emb_u(negative)
        
        ### YOUR CODE HERE ###
        v_c = self.emb_v(center)
        u_o = self.emb_u(outside)
        u_n = self.emb_u(negative)
        #
        # ### YOUR CODE HERE ###
        # Positive sample score
        pos_score = torch.sum(torch.mul(v_c, u_o), dim=1)  # (B,)
        pos_loss = F.logsigmoid(torch.clamp(pos_score, min=-10, max=10))  # (B,)

        # Negative sample scores
        neg_score = torch.bmm(u_n, v_c.unsqueeze(2)).squeeze(2)  # (B, k)
        neg_loss = F.logsigmoid(torch.clamp(-neg_score, min=-10, max=10))  # (B, k)

        # Combine losses
        loss = -torch.sum(pos_loss + torch.sum(neg_loss, dim=1))  # Scalar

        # Hint: torch.clamp the input to F.logsigmoid to avoid numerical underflow/overflow
        ### END YOUR CODE ###

        return loss
    
    def save_embedding(self, id2word, file_name):
        embedding = self.emb_v.weight.cpu().data.numpy()
        with open(file_name, 'w') as f:
            f.write('%d %d\n' % (len(id2word), self.emb_size))
            for wid, w in id2word.items():
                e = ' '.join(map(lambda x: str(x), embedding[wid]))
                f.write('%s %s\n' % (w, e))

### 3. Train and Evaluate

In [ ]:
def train(model, dataloader, optimizer, epochs):
    # Write your own code for this train function
    # You don't need exactly the same arguments

    ### YOUR CODE HERE ###
    pass
    ### END YOUR CODE ###


# Suggested hyperparameters
initial_lr = 0.025
batch_size = 16
emb_size = 50
window_size = 5
k = 10 # the number of negative samples, change with your own choice for better embedding performance
min_count = 1 # because our data is small. If min_count > 1, you should filter out those unknown words from the data in train() function
optimizer = torch.optim.Adam() # or torch.optim.SparseAdam()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR() # or torch.optim.lr_scheduler.StepLR()

# Initialize the corpus and model
corpus = CorpusReader('lunyu_20chapters.txt', min_count)
model = SkipGram(corpus.vocab_size, emb_size)


### Hints: ###
# - If you have cuda-supported GPUs, you can run the training faster by
#   `device = torch.device("cuda" if self.use_cuda else "cpu")`
#   `model.cuda()`
#   You also need to move all tensor data to the same device
# - If you find Inf or NaN in the loss, you can try to clip the gradient usning `torch.nn.utils.clip_grad_norm_`
# - Remember to save the embeddings when training is done

### 4. Plot and Compare Embeddings

Use `sklearn.decomposition.TruncatedSVD` to reduce the dimensionality of the obtained embeddings to 2 and plot the selected words in 2D space.

*Hint*:
- Obtain the embeddings into a numpy array by `model.emb_v.cpu().data.numpy()`
- The word2id dictionary is in `model.word2id`
- If you are trying to load from a saved embedding file, you can use the APIs from `gensim`.
  - For exmaple, `model = gensim.models.KeyedVectors.load_word2vec_format('path/to/file')`
  - Check out the documentation for more details: https://radimrehurek.com/gensim/models/keyedvectors.html

In [ ]:
# Load embeddings
### YOUR CODE HERE ###
pass
### END YOUR CODE ###

In [ ]:
# Truncated SVD
### YOUR CODE HERE ###
pass
### END YOUR CODE ###

In [ ]:
# Plot the following words or other words you are interested in
# You better pick those words that look different in the 2D space compared with the LSA vectors
words = ['学', '习', '曰', '子', '人', '仁']
words_pinyin = ['xue', 'xi', 'yue', 'zi', 'ren1', 'ren2']

### YOUR CODE HERE ###
pass
### END YOUR CODE ###